In [37]:
import tensorflow as tf
from modules.encoder import Encoder

In [39]:
vocab_size = 20000
max_len = 200

(x_train, y_train), (x_test, y_test) = tf.keras.datasets.imdb.load_data(num_words=vocab_size)
print(f'학습 리뷰 개수 : {len(x_train)}')
print(f'검증 리뷰 개수 : {len(x_test)}')

학습 리뷰 개수 : 25000
검증 리뷰 개수 : 25000


In [40]:
x_train = tf.keras.preprocessing.sequence.pad_sequences(x_train, maxlen=max_len)
x_test = tf.keras.preprocessing.sequence.pad_sequences(x_test, maxlen=max_len)

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, SimpleRNN, Embedding

In [41]:
embedding_dim = 32
hidden_units = 32

model = Sequential()
model.add(Embedding(vocab_size, embedding_dim))
model.add(SimpleRNN(hidden_units))
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['acc'])

In [43]:
history = model.fit(x_train, y_train, epochs=5, batch_size=64, validation_split=0.2)

Epoch 1/5
313/313 [==============================] - 10s 32ms/step - loss: 0.4190 - acc: 0.8161 - val_loss: 0.3925 - val_acc: 0.8320
Epoch 2/5
313/313 [==============================] - 9s 30ms/step - loss: 0.2938 - acc: 0.8809 - val_loss: 0.3916 - val_acc: 0.8246
Epoch 3/5
313/313 [==============================] - 14s 44ms/step - loss: 0.2089 - acc: 0.9186 - val_loss: 0.4261 - val_acc: 0.8364
Epoch 4/5
313/313 [==============================] - 22s 70ms/step - loss: 0.1373 - acc: 0.9517 - val_loss: 0.4297 - val_acc: 0.8422
Epoch 5/5
313/313 [==============================] - 21s 66ms/step - loss: 0.0787 - acc: 0.9742 - val_loss: 0.4767 - val_acc: 0.8428


In [44]:
print(model.evaluate(x_test, y_test)[1])

782/782 [==============================] - 10s 13ms/step - loss: 0.5110 - acc: 0.8336
0.8335999846458435
